1. Import Library

In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.3 MB/s eta 0:00:00


In [2]:
import io
import string
import numpy
import random
import sklearn.svm as svm

from sklearn.svm import LinearSVC
from collections import defaultdict
from pyvi import ViTokenizer, ViPosTagger
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from keras.models import save_model
from sklearn.metrics import accuracy_score
import pickle

3. Tagset

In [ ]:
"""
A - Adjective
C - Coordinating conjunction
E - Preposition
I - Interjection
L - Determiner
M - Numeral
N - Common noun
Nc - Noun Classifier
Ny - Noun abbreviation
Np - Proper noun
Nu - Unit noun
P - Pronoun
R - Adverb
S - Subordinating conjunction
T - Auxiliary, modal words
V - Verb
X - Unknown
F - Filtered out (punctuation)
"""

'\nA - Adjective\nC - Coordinating conjunction\nE - Preposition\nI - Interjection\nL - Determiner\nM - Numeral\nN - Common noun\nNc - Noun Classifier\nNy - Noun abbreviation\nNp - Proper noun\nNu - Unit noun\nP - Pronoun\nR - Adverb\nS - Subordinating conjunction\nT - Auxiliary, modal words\nV - Verb\nX - Unknown\nF - Filtered out (punctuation)\n'

In [3]:
# Tạo dictionary cho các loại từ loại và chuyển đổi ngược lại
tagsetDict = {"N" : 1,
              "Np" : 2,
              "Nc" : 3,
              "Nu" : 4,
              "V" : 5,
              "A" : 6,
              "P" : 7,
              "L" : 8,
              "M" : 9,
              "R" : 10,
              "E" : 11,
              "C" : 12,
              "I" : 13,
              "T" : 14,
              "B" : 15,
              "Y" : 16,
              "S" : 17,
              "X" : 18 }
inverseTagsetDict = {tagsetDict[k]:k for k in tagsetDict}

4. Check some of the tagged words

In [4]:
# Khởi tạo các dictionary và list cho việc lưu trữ thông tin từ và bigram

wordBank = defaultdict()
bigramBank = defaultdict()
bigramDict = defaultdict(int)

bigramFreq = {x:[0]*18 for x in tagsetDict}
mostCommonBigrams = defaultdict()

prevParts = None

# Đọc dữ liệu huấn luyện và kiểm tra
f_tagged_train = io.open("vi_train.txt", encoding='utf-8').readlines()
f_tagged_test = io.open("vi_test.txt", encoding='utf-8').readlines()

train = f_tagged_train[:12100]
test = f_tagged_test[:10000]
print(train[:10])
print(test[:10])

['\n', 'Trên/E đường/N đi/V ,/, có/V một/M lần/N xe/N cô/N suýt/R rơi/V xuống/R vực/N ở/E đèo/N Ngoạn_Mục/Np ./.\n', '\n', 'Trong/E một/M trận/N đánh/V ác_liệt/A bên/N thành/N cổ/N Quảng_Trị/Np ,/, một/M loạt/N đạn/N pháo/N của/E kẻ_thù/N đã/R rơi/V trúng/A chỗ/N chiến_sĩ/N thông_tin/N Nguyễn_Văn_Thạc/Np .../... Hôm/N đó/P ngày/N 30/M -/- 7/M -/- 1972/M ./.\n', '\n', 'Sau/E khi/N H./Ny “/“ AK/Ny ”/” bị/V xộ/V khám/V ,/, Hoàng/Np đã/R hoàn_lương/V ,/, bây_giờ/P đang/R phụ/V việc/N cho/E gia_đình/N Quân/Np ./.\n', '\n', 'Hà/Np ,/, 21/M tuổi/N -/- làm/V nghề/N hớt_tóc/V ,/, khai/V :/: “/“ Ban_đầu/N nghe/V mấy/L ảnh/N nói/V thuốc/N lắc/V không/R nghiện/V nên/C em/N uống/V thử/V ./.\n', '\n', 'Theo/E anh/Nc Thông/Np ,/, người/N già/A ở/E Đức/Np thường/R sống/V một_mình/Ny và/C sống/V với/E những/L kỷ_vật/N của/E riêng/A mình/P mang/V về/R từ/E những/L chuyến/N du_lịch/V hay/C những/L tặng_vật/N của/E cả/P một/M đời/N ./.\n']
['Trên/E đường/N xuất_hiện/V nhiều/A cặp/N cha/N -/- con/N ,/, mẹ/

In [5]:
print(len(f_tagged_train))
print(len(f_tagged_test))

17069
2151


In [6]:
# Phân tích dữ liệu huấn luyện để tạo từ loại và bigrams
for line in train:
  l_split = line.split()
  for i, w in enumerate(l_split):

    parts = w.split('/')
    if i >=1:
      prevParts = l_split[i-1].split('/')
    if len(parts) == 1 or \
      parts[1] not in tagsetDict:
      continue

    if i >=2 and prevParts[1] in string.punctuation:
      prevParts = l_split[i - 1].split('/')
    word = parts[0].lower()
    pos = parts[1]

    if i >= 1:
      prevWord = prevParts[0]
      prevPos = prevParts[1]
      bigramBank[word] = (pos, prevWord, prevPos)
      bigramDict[(pos, prevPos)] += 1
    if word not in wordBank:
      wordBank[word] = [pos]
    else:
      wordBank[word] +=[pos]

In [7]:
# Tạo các list chứa bi-grams, tri-grams, qua-grams từ dữ liệu
bi_grams = []
tri_grams = []
qua_grams = []

for sentence in set(wordBank):
  temp = 0
  for s in list(sentence):
    if s == '_':
      temp+=1
    if temp == 0:
      bi_grams.append(sentence)
    elif temp == 2:
      tri_grams.append(sentence)
    elif temp == 4:
      qua_grams.append(sentence)



In [ ]:
print(bi_grams)

['tiên_tiến', 'tiên_tiến', 'tiên_tiến', 'tiên_tiến', 'làm_lụng', 'làm_lụng', 'làm_lụng', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'bến_xe', 'bến_xe', 'bến_xe', 'chật_vật', 'chật_vật', 'chật_vật', 'chật_vật', 'khép', 'khép', 'khép', 'khép', 'Lang_thang', 'Lang_thang', 'Lang_thang', 'Lang_thang', 'nên', 'nên', 'nên', '10%', '10%', '10%', 'áp_suất', 'áp_suất', 'điều_kiện', 'điều_kiện', 'điều_kiện', 'điều_kiện', 'kê', 'kê', 'Khu_ủy', 'Khu_ủy', 'Khu_ủy', 'hớp', 'hớp', 'hớp', 'Vĩnh_Phúc', 'Vĩnh_Phúc', 'Vĩnh_Phúc', 'Vĩnh_Phúc', 'Tắt', 'Tắt', 'Tắt', 'Tay', 'Tay', 'Tay', 'French_Style', 'French_Style', 'French_Style', 'French_Style', 'French_Style', 'French_Style', 'lên_tiếng', 'lên_tiếng', 'lên_tiếng', 'Xin_lỗi', 'Xin_lỗi', 'Xin_lỗi', 'Dọn', 'Dọn', 'Dọn', 'hẳn_hòi', 'hẳn_hòi', 'hẳn_hòi', 'long_bào', 'long_bào', 'long_bào', 'long_bào', 'pháp_lệnh', 'pháp_lệnh', 'pháp_lệnh', 'pháp_lệnh', 'lia', 'lia', 'lia', '93.000', '93.000', '93.000', '93.000', '93.000', '93.000', 'Vì_vậy', 'Vì_vậy'

In [ ]:
print(tri_grams)

['Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Phó_Kỳ_Phương', 'Mai_Hoàng_Mỹ_Trang', 'Mai_Hoàng_Mỹ_Trang', 'Mai_Hoàng_Mỹ_Trang', 'siêu_người_mẫu', 'siêu_người_mẫu', 'siêu_người_mẫu', 'siêu_người_mẫu', 'huấn_luyện_viên', 'huấn_luyện_viên', 'huấn_luyện_viên', 'huấn_luyện_viên', 'huấn_luyện_viên', 'Đài_truyền_hình', 'Đài_truyền_hình', 'Đài_truyền_hình', 'Đài_truyền_hình', 'Đài_truyền_hình', 'Long_Hổ_Hội', 'Long_Hổ_Hội', 'Long_Hổ_Hội', 'Long_Hổ_Hội', 'Khu_chế_xuất', 'Khu_chế_xuất', 'Khu_chế_xuất', 'Khu_chế_xuất', 'Khu_chế_xuất', 'Tiết_Thủy_Sơn', 'Tiết_Thủy_Sơn', 'Tiết_Thủy_Sơn', 'Tiết_Thủy_Sơn', 'nhà_sản_xuất', 'nhà_sản_xuất', 'nhà_sản_xuất', 'nhà_sản_xuất', 'nhà_sản_xuất', 'Mỹ_Thành_Nam', 'Mỹ_Thành_Nam', 'Mỹ_Thành_Nam', 'Mỹ_Thành_Nam', 'Phan_Thị_Bi', 'Phan_Thị_Bi', 'Phan_Thị_Bi', 'Lý_Thái_Tổ', 'Lý_Thái_Tổ', 'Lý_Thái_Tổ', 'máy_phát_điện', 'máy_phát_điện', 'máy_phát_điện', 'máy_phát_điện', 'máy_phát_điện', 'Trương_Tấn_Nghĩa', 'Trương_

In [ ]:
print(qua_grams)

['Tuệ_Tĩnh_Đường_Diệu_Đế', 'Tuệ_Tĩnh_Đường_Diệu_Đế', 'Tuệ_Tĩnh_Đường_Diệu_Đế', '15_-_11_-_1971', '15_-_11_-_1971', '15_-_11_-_1971', '15_-_11_-_1971', '15_-_11_-_1971']


**5. SVM Classification**

In [8]:
# Tìm các bigram phổ biến nhất cho mỗi từ loại
for k in bigramFreq:
  maxFreq = 0
  maxPos = 'Np'
  for i, x in enumerate(bigramFreq[k]):
    if x > maxFreq:
      maxFreq = x
      maxPos = inverseTagsetDict[i]
    mostCommonBigrams[k] = maxPos

In [9]:
def Viterbi_rule_based(word,wordIdx, lineSize, line):
    """
    Hàm này tạo vector đặc trưng cho từng từ dựa trên quy tắc Viterbi.

    Parameters:
        word (str): Từ hiện tại.
        wordIdx (int): Vị trí của từ trong câu.
        lineSize (int): Độ dài của câu.
        line (list): Danh sách các từ trong câu.

    Returns:
        list: Vector đặc trưng cho từ.
    """
    feat = [1]
    sentPercent = float(wordIdx)/float(lineSize)
    feat.append(sentPercent)

    if word[0].isupper() and wordIdx != 0:
        feat.append(1)
    else:
        feat.append(0)

    posIdx_array = ([0] * len(tagsetDict))
    posSet = []
    if word in wordBank:
        posSet = wordBank[word]
    else:
        if wordIdx == 0:
            posSet = list(tagsetDict.keys())[0]
            posIdx_array[tagsetDict[posSet]] = 1
            return feat + posIdx_array + [0]
        else:
            prevWord = line[wordIdx-1]
            if prevWord in wordBank:
                prevPos = wordBank[prevWord]
                maxPos = mostCommonBigrams[prevPos]
                if prevPos == "E":
                    maxPos = 3
                posIdx_array[tagsetDict[maxPos]] = 1
                feat += posIdx_array + [tagsetDict[maxPos]]
                return feat
            else:
                posSet = list(tagsetDict.keys())[0]
                posIdx_array[tagsetDict[posSet]] = 1
                feat += posIdx_array + [0]
                return feat

    for pos in posSet:
        posIdx = tagsetDict[pos]
        posIdx_array[posIdx] += 1.0/len(wordBank[word])
    feat += (posIdx_array) + [0]
    return feat

In [10]:
y_train = []
X_train = []
for line in train:
  l_split = line.split()
  for i, w in enumerate(l_split):
    parts = w.split('/')
    word = parts[0].lower()
    len_line = len(l_split)

    if len(parts) == 1 or parts[1] not in tagsetDict:
      continue
    y_train.append(wordBank[word][0])
    X_train.append(Viterbi_rule_based(word, i, len_line, l_split))

In [11]:
print (len(X_train))
print(len(y_train))
for i in range(10):
  print(i, ':', X_train[i])
print(y_train[:10])


115791
115791
0 : [1, 0.0, 0, 0, 0.04387990762124712, 0, 0, 0, 0, 0.1454965357967666, 0, 0, 0, 0, 0.8106235565819839, 0, 0, 0, 0, 0, 0, 0]
1 : [1, 0.058823529411764705, 0, 0, 0.9887218045112836, 0.011278195488721804, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2 : [1, 0.11764705882352941, 0, 0, 0, 0, 0, 0, 0.900140646976075, 0, 0, 0, 0, 0.09423347398030948, 0, 0, 0, 0.005625879043600563, 0, 0, 0, 0]
3 : [1, 0.23529411764705882, 0, 0, 0, 0.010703363914373088, 0, 0, 0.9709480122323952, 0, 0, 0, 0, 0.005351681957186545, 0.010703363914373088, 0, 0, 0.0022935779816513763, 0, 0, 0, 0]
4 : [1, 0.29411764705882354, 0, 0, 0.0006839945280437756, 0, 0, 0, 0, 0, 0, 0.0006839945280437756, 0.9986320109439333, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5 : [1, 0.35294117647058826, 0, 0, 0.9786324786324764, 0, 0, 0, 0.021367521367521368, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6 : [1, 0.4117647058823529, 0, 0, 0.9916666666666656, 0, 0, 0, 0.008333333333333333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7 : [1, 0.47058

In [12]:
X_test = []
y_test = []
for line in test:
  l_split = line.split()
  for i, w in enumerate(l_split):
    len_line = len(l_split)
    parts = w.split('/')
    if len(parts) == 1 or parts[1] not in tagsetDict:
      continue
    word = parts[0].lower()
    pos = parts[1]
    if word in string.punctuation or word == ':.':
      continue
    X_test.append(Viterbi_rule_based(word, i, len_line, l_split))
    y_test.append(pos)


**6. Test accuracy on subset of test data**

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
model = OneVsRestClassifier(LinearSVC(random_state=0))

for epoch in range(5):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)

    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)

    train_precision = precision_score(y_train, y_pred_train, average='micro')
    val_precision = precision_score(y_val, y_pred_val, average='micro')

    train_recall = recall_score(y_train, y_pred_train, average='micro')
    val_recall = recall_score(y_val, y_pred_val, average='micro')

    print(f"Epoch {epoch+1}:")
    print(f"  Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"  Train Precision: {train_precision:.4f}, Validation Precision: {val_precision:.4f}")
    print(f"  Train Recall: {train_recall:.4f}, Validation Recall: {val_recall:.4f}")

with open('postagging_SVM_Vi.pkl', 'wb') as f:
    pickle.dump(model, f)


Epoch 1:
  Train Accuracy: 0.9260, Validation Accuracy: 0.9267
  Train Precision: 0.9260, Validation Precision: 0.9267
  Train Recall: 0.9260, Validation Recall: 0.9267
Epoch 2:
  Train Accuracy: 0.9260, Validation Accuracy: 0.9267
  Train Precision: 0.9260, Validation Precision: 0.9267
  Train Recall: 0.9260, Validation Recall: 0.9267
Epoch 3:
  Train Accuracy: 0.9260, Validation Accuracy: 0.9267
  Train Precision: 0.9260, Validation Precision: 0.9267
  Train Recall: 0.9260, Validation Recall: 0.9267
Epoch 4:
  Train Accuracy: 0.9260, Validation Accuracy: 0.9267
  Train Precision: 0.9260, Validation Precision: 0.9267
  Train Recall: 0.9260, Validation Recall: 0.9267
Epoch 5:
  Train Accuracy: 0.9260, Validation Accuracy: 0.9267
  Train Precision: 0.9260, Validation Precision: 0.9267
  Train Recall: 0.9260, Validation Recall: 0.9267


In [14]:
with open('postagging_SVM_Vi.pkl', 'rb') as f:
    model = pickle.load(f)

y_pred_test = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='micro')
test_recall = recall_score(y_test, y_pred_test, average='micro')

print("\nFinal Evaluation on Test Set:")
print(f"  Test Accuracy: {test_accuracy:.4f}")
print(f"  Test Precision: {test_precision:.4f}")
print(f"  Test Recall: {test_recall:.4f}")


Final Evaluation on Test Set:
  Test Accuracy: 0.8703
  Test Precision: 0.8703
  Test Recall: 0.8703


In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

           A       0.77      0.90      0.83      1292
           B       1.00      0.14      0.24        22
           C       0.83      0.93      0.88       832
           E       0.92      0.83      0.87      1643
           I       0.33      0.75      0.46         4
           L       0.97      0.93      0.95       436
           M       0.90      0.99      0.94       747
           N       0.95      0.82      0.88      6784
          Nc       0.48      0.80      0.60       354
          Np       0.75      0.93      0.83       710
          Nu       0.82      0.87      0.85       107
           P       0.98      0.94      0.96       996
           R       0.88      0.82      0.85      1768
           S       0.42      0.62      0.50        13
           T       0.27      0.53      0.36        73
           V       0.85      0.92      0.88      4367
           Y       0.00      0.00      0.00         1

    accuracy              

In [16]:
with open('postagging_SVM_Vi.pkl', 'rb') as f:
    model = pickle.load(f)

f_iot_test = io.open("IOT_test1.txt", encoding='utf-8').readlines()
iot_test = f_iot_test[:100]

X_iot_test = []
y_iot_test = []
for line in iot_test:
  l_split = line.split()
  for i, w in enumerate(l_split):
    len_line = len(l_split)
    parts = w.split('/')
    if len(parts) == 1:
      continue
    word = parts[0].lower()
    pos = parts[1]
    if word in string.punctuation or word == ':.':
      continue
    try:
      X_iot_test.append(Viterbi_rule_based(word, i, len_line, line))
      y_iot_test.append(pos)
    except:
      continue

y_pred_test = model.predict(X_iot_test)
test_accuracy = accuracy_score(y_iot_test, y_pred_test)
test_precision = precision_score(y_iot_test, y_pred_test, average='micro')
test_recall = recall_score(y_iot_test, y_pred_test, average='micro')

print("\nFinal Evaluation on IOT Test Set:")
print(f"  Test Accuracy: {test_accuracy:.4f}")
print(f"  Test Precision: {test_precision:.4f}")
print(f"  Test Recall: {test_recall:.4f}")


Final Evaluation on IOT Test Set:
  Test Accuracy: 0.8286
  Test Precision: 0.8286
  Test Recall: 0.8286


# ***7. Using the Longest Matching algorithm***

In [17]:
import unicodedata as ud
import re

def syllablize(sentence):
  """
    Hàm này chia câu thành các âm tiết.

    Parameters:
        sentence (str): Câu cần chia.

    Returns:
        list: Danh sách các âm tiết trong câu.
    """
  word = '\w+'
  non_word = '[^\w\s]'
  digits = '\d+([\.,_]\d+)+'

  patterns = []
  patterns.extend([word, non_word, digits])
  patterns = f"({'|'.join(patterns)})"

  sentence = ud.normalize('NFC', sentence)
  tokens = re.findall(patterns, sentence, re.UNICODE)
  return [token[0] for token in tokens]

In [18]:
def longest_matching(sentence, bi_grams, tri_grams):
  """
    Hàm này tìm các từ ghép dài nhất trong câu.

    Parameters:
        sentence (str): Câu cần kiểm tra.
        bi_grams (list): Danh sách các bi-grams.
        tri_grams (list): Danh sách các tri-grams.

    Returns:
        list: Danh sách các từ ghép dài nhất trong câu.
    """
  syllables = syllablize(sentence)
  syl_len = len(syllables)

  curr_id = 0
  word_list = []
  done = False

  while (curr_id < syl_len) and (not done):
    curr_word = syllables[curr_id]
    if curr_id >= syl_len - 1:
      word_list.append(curr_word)
      done = True
    else:
      next_word = syllables[curr_id + 1]
      pair_word = " ".join([curr_word.lower(), next_word.lower()])
      if curr_id >=(syl_len - 2):
        if pair_word in bi_grams:
          word_list.append("_".join([curr_word, next_word]))
          curr_id += 2
        else:
          word_list.append(curr_word)
          curr_id += 1
      else:
        next_next_word = syllables[curr_id + 2]
        triple_word = ' '.join([pair_word, next_next_word.lower()])
        if triple_word in tri_grams:
          word_list.append('_'.join([curr_word, next_word, next_next_word]))
          curr_id += 3
        elif pair_word in bi_grams:
          word_list.append('_'.join([curr_word, next_word]))
          curr_id += 2
        else:
          word_list.append(curr_word)
          curr_id += 1
  return word_list

8. Conver into String

In [19]:
def toString(wl, model):
  """
    Hàm này chuyển đổi danh sách các từ thành chuỗi.

    Parameters:
        wl (list): Danh sách các từ cần chuyển đổi.

    Returns:
        str: Chuỗi kết quả.
    """
  wl = longest_matching(wl, bi_grams, tri_grams)
  X = []
  A = []
  text = ''
  for i in set(wl):
    if i == '.':
      A = './.'
    else:
      X.append(Viterbi_rule_based(i, 1, 1, 1))
      A = str(model.predict(X))
  for i in range(len(wl)):
    text += wl[i]
    text +='/'
    text +=str(A[2])
    text += ' '

  return wl[i], str(A[2])

10. Check how a sentence is tagged by the POS tagger

In [21]:
if __name__ == "__main__":
    #wr = input('Text: ')
    with open('postagging_SVM_Vi.pkl', 'rb') as f:
      model = pickle.load(f)
    wr ="""Mở cửa nhà vệ sinh""".lower()
    print(wr)
    result = {'N': [], 'V': []}
    wl = ViTokenizer.tokenize(wr)
    wl = wl.split()
    for i in wl:
      try:
          word, pos =  toString(i, model)
          result[pos].append(word)
      except Exception:
          #print(i,"X",sep='/',end='')
          #print(' ',end='')
          pass
    print(result)

mở cửa nhà vệ sinh
{'N': ['cửa', 'nhà'], 'V': ['mở', 'vệ_sinh']}
